In [ ]:
# Install required packages
!pip install transformers evaluate nltk rouge-score --quiet

# Imports
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import evaluate
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer


ModuleNotFoundError: No module named 'evaluate'

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()


In [ ]:
# generate the sentence

prompt = "The future of artificial intelligence is"

input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generate text with top-p sampling
output_ids = model.generate(
    input_ids=input_ids,
    max_length=30,
    do_sample=True,
    top_p=0.9,
    top_k=50,
    temperature=0.8
)

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("🧠 Generated Output:\n", generated_text)


In [ ]:
# reference sentence

# Reference sentences written manually
references = [
    "The future of artificial intelligence is full of potential and ethical challenges.",
    "Artificial intelligence will transform industries in the future."
]


In [ ]:
# BLEU

from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

candidate_tokens = word_tokenize(generated_text.lower())
reference_tokens = [word_tokenize(ref.lower()) for ref in references]

bleu_score = sentence_bleu(reference_tokens, candidate_tokens,
                           smoothing_function=SmoothingFunction().method1)
print("🎯 BLEU Score:", round(bleu_score, 4))


In [ ]:
# Rouge

from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# ROUGE requires string format
for ref in references:
    rouge_scores = scorer.score(ref, generated_text)
    print(f"\n🔍 ROUGE Scores vs: {ref}")
    for key, value in rouge_scores.items():
        print(f"{key}: Precision={value.precision:.4f}, Recall={value.recall:.4f}, F1={value.fmeasure:.4f}")


In [ ]:
# meteor

meteor_scores = [meteor_score([ref], generated_text) for ref in references]
print("\n📐 METEOR Scores:")
for i, score in enumerate(meteor_scores):
    print(f"Reference {i+1}: {round(score, 4)}")

# Average Meteor score
print("📊 Avg METEOR:", round(sum(meteor_scores)/len(meteor_scores), 4))
